In [21]:
import pandas as pd
import re
import requests
from dateutil.relativedelta import relativedelta
from datetime import datetime

data_raw = pd.read_csv("data_raw/participantes.csv", sep = ";").drop(axis=1, labels=["Unnamed: 0"])

Unindo colunas de nascimento e renomeando

In [22]:
cols = ['Data de nascimento', 'Idade', 'Data de Nascimento']
data = data_raw.assign(
    data_nascimento = lambda x: x['Data de nascimento'].fillna("") +
                                x['Data de Nascimento'].fillna("") +
                                x['Idade'].fillna("")
).drop(cols, axis = 1).drop("Ref.", axis = 1)
data = data.rename(columns = {"Participante": 'nome', "Ocupação": "ocupacao", "Resultado":"resultado"})

In [98]:
def remover_nota(data):
    
    try:
        data = str(data)
        data = re.sub("\[(.*)\]", "", data)
        data = re.sub("\?", "", data)
        data = datetime.strptime(data, "%d/%m/%Y")
        return data
    except:
        return None    

data = data.assign(
    data_nascimento = data.apply(lambda x: remover_nota(x['data_nascimento']), axis = 1)
)

Corrigindo localidade de origem

In [99]:
data_local = data.assign(
    municipio = lambda x: x["Origem"].str.split(",").str[0],
    estado = lambda x: x["Origem"].str.split(",").str[1]
).drop(["Origem"], axis = 1)

data_local['estado'] = data_local['estado'].fillna("Exterior")

Corrigindo Resultado

In [100]:
num_participantes = data_local.groupby(["edicao"]).agg(
    n_participantes = ("edicao", "count")
)

participantes_edicao = data_local.join(num_participantes, on ='edicao').assign(
    resultado = lambda x: x['resultado'].str.split("em").str[0]
)

In [101]:
def arrumar_classificacao(row):
    # texto, n_participantes

    if any(i in row['resultado'] for i in ["Vencedor", "Vencedora"]):
        row['resultado'] = re.sub("Ven.*", "1", row['resultado'])
    elif any(i in row['resultado'] for i in ["eliminado", "eliminada"]):
        lugar = re.sub("ª eli.*|º eli.*", "", row['resultado'])
        lugar = row['n_participantes'] + 1 - int(lugar)
        row['resultado'] = str(lugar)
    elif "lugar" in row['resultado']: 
        row['resultado'] = row['resultado'][0]
    else:
        row['resultado'] = None 
    
    return row['resultado']

participantes_edicao['classificacao'] = participantes_edicao.apply(arrumar_classificacao, axis = 1)
participantes_edicao = participantes_edicao.drop(['resultado'], axis = 1)

- Gênero #Não rodar esse codigo

In [216]:
# def genero(row):
#     key = "usei o gender-api"
#     primeiro_nome = row['nome'].split(" ")[0]
#     url = f"https://gender-api.com/get?name={primeiro_nome}&key={key}"
#     gender_api = requests.get(url)
#     gender_json = eval(gender_api.text)
#     gender = gender_json["gender"]

#     return gender

# participantes_edicao['genero'] = participantes_edicao.apply(genero, axis = 1)

In [102]:
genero = pd.read_csv("data/genero.csv", sep = ';')
participantes_edicao = participantes_edicao.merge(genero, how='left', on="nome" )
participantes_edicao.to_csv("data_raw/participantes_genero.csv", sep = ";", index = False)
participantes_genero = pd.read_csv("data_raw/participantes_genero.csv", sep = ";")

- Idade quando o programa começou

In [103]:
from wikitable import wikitable

url = "https://pt.wikipedia.org/wiki/Big_Brother_Brasil"
temporadas_raw = wikitable(url)[2]

In [104]:
temporadas_raw.columns = temporadas_raw.columns.droplevel()

In [105]:
estreias = temporadas_raw[["Temporada.1", "Estreia da temporada"]].rename({"Temporada.1": "edicao", "Estreia da temporada": "estreia"}, axis = 1)

estreias['edicao'] = estreias['edicao'].str.split(" ").str[-1]
estreias['estreia'] = estreias['estreia'].str.replace(" de ", "/").str.replace("janeiro", "01").str.replace("maio", "05")

estreias['edicao'] = pd.to_numeric(estreias['edicao'])
participantes_estreia = participantes_genero.merge(estreias, on="edicao", how='left')

participantes_estreia[['data_nascimento', 'estreia']] = participantes_estreia[['data_nascimento', 'estreia']].apply(pd.to_datetime, infer_datetime_format=True)

participantes_estreia = participantes_estreia.assign(
    idade_participacao = lambda x: x['estreia'].dt.year - x['data_nascimento'].dt.year
).drop(labels=['estreia', 'n_participantes'], axis = 1)

In [107]:
participantes_estreia = participantes_estreia[['nome', 'genero', "data_nascimento", "idade_participacao", "ocupacao", "municipio", "estado", "edicao", "classificacao"]]

In [108]:
participantes_estreia.to_csv("data/dados.csv", sep =';', index = False)